## Search Parameters

Each Facebook post is classified by whether it contains a carpool(s), and if so, whether it is a driving or searching carpool(s), the origin(s), the destination(s) and the date of the carpool(s).

## Data Preprocessing:

In order to effectively search a Facebook post, it must be "cleaned" first. This is done via spellcheck, lemmatizing, removing stopwords (except for the words "to" and "from"), converting specific names to a standard form and converting certain punctuation symbols to words (for example "->" is converted to "to").

## Driving, Searching and Other Classification

Discovering whether a post is a "driving" post (a carpool is being offered), a "searching" post (a carpool is being requested) or an "other" post (a completetly unrelated post) is a trinary classification problem. As such, a supervised learning algorthm is used.

The training data consists 1366 manually classified posts. Below is a preview of the data where "post_type" is the classification and "stage_3" is the processed text.

In [1]:
import pandas as pd
from db import engine
derived_posts = pd.read_sql_query('SELECT * from derived_posts', con=engine)

data = pd.read_sql_query('SELECT A.post_id, A.post_type, A.route_count, B.stage_3 FROM manual_posts A LEFT JOIN derived_posts B ON (A.post_id = B.post_id) ', con=engine)
data[["post_type", "stage_3"]].head()

,post_type,stage_3
0,d,"driving,waterloo,to,markham,sunday,may,27,at,3..."
1,d,"driving,toronto,to,waterloo,sunday,may,27th,at..."
2,d,"leaving,waterloo,to,toronto,7pm,today"
3,d,"driving,scarborough,to,waterloo,at,8am,monday,..."
4,d,"driving,fairview,mall,to,waterloo,sunday,may,2..."


In [2]:
import nltk
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV

def split_by_comma(x):
    return x.split(",")

The classifier used is the random forest classifier, due its reputation of being robust to overfitting. A grid search cross validation is performed to find the optimal parameters and the optimal vectorizor for the text.  The grid search is performed over n_estimators = 50, 150, 300; max_depth = 60, 90, None; bootstrap = True, False; and the TFIDF, count and 2-gram vectorizers.

In [3]:
rf = RandomForestClassifier()
param = {'n_estimators': [50, 150,300], 'max_depth': [60,90,None], 'bootstrap': ['True','False']}
gs = GridSearchCV(rf, param, cv = 5, n_jobs = -1, return_train_score= True)

Using the TFIDF vectorizer:

In [4]:
tfidf_vect = TfidfVectorizer(analyzer = split_by_comma)
X_tfidf = tfidf_vect.fit_transform(data["stage_3"])
X_tfidf_feat = pd.DataFrame(X_tfidf.toarray())
gs_fit_tfidf = gs.fit(X_tfidf_feat, data["post_type"])
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(gs_fit_tfidf.cv_results_).sort_values(by = "rank_test_score")[["params","mean_test_score", "std_test_score", "mean_fit_time", "std_fit_time", "mean_score_time", "std_score_time"]]

,params,mean_test_score,std_test_score,mean_fit_time,std_fit_time,mean_score_time,std_score_time
10,"{'bootstrap': 'False', 'max_depth': 60, 'n_estimators': 150}",0.975842,0.010438,0.535766,0.118265,0.017554,0.001352
1,"{'bootstrap': 'True', 'max_depth': 60, 'n_estimators': 150}",0.974378,0.011762,0.557508,0.095647,0.018151,0.001163
4,"{'bootstrap': 'True', 'max_depth': 90, 'n_estimators': 150}",0.973646,0.010113,0.600394,0.039336,0.018950,0.001092
13,"{'bootstrap': 'False', 'max_depth': 90, 'n_estimators': 150}",0.973646,0.015869,0.498069,0.135294,0.017952,0.001092
14,"{'bootstrap': 'False', 'max_depth': 90, 'n_estimators': 300}",0.972914,0.007480,1.022864,0.110984,0.060039,0.019790
11,"{'bootstrap': 'False', 'max_depth': 60, 'n_estimators': 300}",0.972914,0.010444,1.159101,0.227544,0.064029,0.026987
17,"{'bootstrap': 'False', 'max_depth': None, 'n_estimators': 300}",0.972182,0.009634,1.092479,0.169919,0.052660,0.009301
2,"{'bootstrap': 'True', 'max_depth': 60, 'n_estimators': 300}",0.971449,0.012075,1.086494,0.214378,0.056849,0.014397
5,"{'bootstrap': 'True', 'max_depth': 90, 'n_estimators': 300}",0.971449,0.012926,1.055578,0.125692,0.072806,0.017262
7,"{'bootstrap': 'True', 'max_depth': None, 'n_estimators': 150}",0.971449,0.013303,0.521007,0.170913,0.018750,0.001465


Using the count vectorizer:

In [5]:
count_vect = CountVectorizer(analyzer = split_by_comma)
X_count = count_vect.fit_transform(data["stage_3"])
X_count_feat= pd.DataFrame(X_count.toarray())
gs_fit_count = gs.fit(X_count_feat, data["post_type"])
pd.DataFrame(gs_fit_count.cv_results_).sort_values(by = "rank_test_score")[["params","mean_test_score", "std_test_score", "mean_fit_time", "std_fit_time", "mean_score_time", "std_score_time"]]

,params,mean_test_score,std_test_score,mean_fit_time,std_fit_time,mean_score_time,std_score_time
9,"{'bootstrap': 'False', 'max_depth': 60, 'n_estimators': 50}",0.980234,0.006321,0.215823,0.068382,0.007978,0.001261
11,"{'bootstrap': 'False', 'max_depth': 60, 'n_estimators': 300}",0.977306,0.007751,1.070338,0.230143,0.075200,0.023522
13,"{'bootstrap': 'False', 'max_depth': 90, 'n_estimators': 150}",0.976574,0.005871,0.597003,0.127303,0.018950,0.001669
2,"{'bootstrap': 'True', 'max_depth': 60, 'n_estimators': 300}",0.975842,0.003659,0.893611,0.073811,0.043685,0.007527
5,"{'bootstrap': 'True', 'max_depth': 90, 'n_estimators': 300}",0.975842,0.009650,0.910566,0.180324,0.070014,0.023889
14,"{'bootstrap': 'False', 'max_depth': 90, 'n_estimators': 300}",0.975842,0.008155,1.026256,0.149465,0.064427,0.034499
17,"{'bootstrap': 'False', 'max_depth': None, 'n_estimators': 300}",0.975110,0.005802,0.947466,0.211509,0.046875,0.013307
1,"{'bootstrap': 'True', 'max_depth': 60, 'n_estimators': 150}",0.975110,0.006225,0.493879,0.108622,0.023937,0.009031
4,"{'bootstrap': 'True', 'max_depth': 90, 'n_estimators': 150}",0.975110,0.006638,0.450794,0.105920,0.017554,0.001018
7,"{'bootstrap': 'True', 'max_depth': None, 'n_estimators': 150}",0.975110,0.008707,0.500062,0.114804,0.024136,0.010896


Using the 2-gram vectorizer.

In [6]:
#putting tockenized sentence back into a sentence with spaces for ngrams
data_ngram = data["stage_3"].apply(lambda x: x.replace(","," "))
gram2_vect = CountVectorizer(ngram_range = (2,2))
X_gram2 = gram2_vect.fit_transform(data_ngram)
X_gram2_feat= pd.DataFrame(X_gram2.toarray())
gs_fit_gram2 = gs.fit(X_gram2_feat, data["post_type"])
pd.DataFrame(gs_fit_gram2.cv_results_).sort_values(by = "rank_test_score")[["params","mean_test_score", "std_test_score", "mean_fit_time", "std_fit_time", "mean_score_time", "std_score_time"]]

,params,mean_test_score,std_test_score,mean_fit_time,std_fit_time,mean_score_time,std_score_time
4,"{'bootstrap': 'True', 'max_depth': 90, 'n_estimators': 150}",0.956808,0.018314,3.068197,0.615909,0.118284,0.026765
17,"{'bootstrap': 'False', 'max_depth': None, 'n_estimators': 300}",0.956076,0.012339,6.344037,1.439371,0.228588,0.107978
11,"{'bootstrap': 'False', 'max_depth': 60, 'n_estimators': 300}",0.955344,0.016146,6.198428,1.031843,0.301794,0.062833
5,"{'bootstrap': 'True', 'max_depth': 90, 'n_estimators': 300}",0.955344,0.011522,6.470898,1.017932,0.261103,0.060348
14,"{'bootstrap': 'False', 'max_depth': 90, 'n_estimators': 300}",0.953880,0.011582,6.572427,0.864169,0.291821,0.080401
8,"{'bootstrap': 'True', 'max_depth': None, 'n_estimators': 300}",0.953880,0.013867,6.584395,1.157486,0.288627,0.076094
2,"{'bootstrap': 'True', 'max_depth': 60, 'n_estimators': 300}",0.953880,0.014055,6.302748,1.028051,0.312365,0.104313
16,"{'bootstrap': 'False', 'max_depth': None, 'n_estimators': 150}",0.953148,0.012423,3.130630,0.281382,0.116090,0.026647
12,"{'bootstrap': 'False', 'max_depth': 90, 'n_estimators': 50}",0.953148,0.011982,1.622660,0.350298,0.022142,0.004434
13,"{'bootstrap': 'False', 'max_depth': 90, 'n_estimators': 150}",0.953148,0.013836,3.196853,0.738712,0.124269,0.034176


The mean test score score is essentially 97% for both the TFIDF and count vectorizers and 95% for the 2-gram vectorizer. Due to the negligable difference in accuracy and efficiency, the simplest model (count vectorizer with n_estimators = 60, max_depth = 10 and boostrap = True) is chosen.

In [7]:
model = RandomForestClassifier(n_estimators = 60, max_depth = 10, n_jobs=-1).fit(X_count_feat,data["post_type"])

Due to significant imbalance in the class distribution (in particular, there is a lack of "other" posts as seen in the histogram below), it is important to check where the model is failing. 

In [8]:
data["post_type"].value_counts().plot(kind="bar")

Below is a list displaying the number of times a "searching", "other" and "driving" post was misclassified.

In [9]:
data[model.predict(X_count_feat) != data["post_type"]]["post_type"].value_counts()

o    8
s    7
d    5
Name: post_type, dtype: int64

The percentage of misclassified "other" posts is

In [10]:
8/sum(data["post_type"] == "o")

0.6153846153846154

So the model has trouble classifying "other" posts.

One way to solve this issue might be to try an undersampling method via a balanced random forest classifier. The same gridsearch crossvalidation as before (except with a count vectorizer and a balanced random forest classifier) is run.

In [11]:
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier()
gs = GridSearchCV(brf, param, cv = 5, n_jobs = -1, return_train_score= True)
gs_fit_count = gs.fit(X_count_feat, data["post_type"])
pd.DataFrame(gs_fit_count.cv_results_).sort_values(by = "rank_test_score")[["params","mean_test_score", "std_test_score", "mean_fit_time", "std_fit_time", "mean_score_time", "std_score_time"]]

,params,mean_test_score,std_test_score,mean_fit_time,std_fit_time,mean_score_time,std_score_time
8,"{'bootstrap': 'True', 'max_depth': None, 'n_estimators': 300}",0.943631,0.023546,0.963424,0.020095,0.029720,0.001163
2,"{'bootstrap': 'True', 'max_depth': 60, 'n_estimators': 300}",0.935578,0.009347,0.849528,0.006226,0.025732,0.001323
11,"{'bootstrap': 'False', 'max_depth': 60, 'n_estimators': 300}",0.935578,0.018848,0.883039,0.035880,0.025731,0.001597
7,"{'bootstrap': 'True', 'max_depth': None, 'n_estimators': 150}",0.933382,0.032809,0.530980,0.063094,0.026928,0.020730
1,"{'bootstrap': 'True', 'max_depth': 60, 'n_estimators': 150}",0.932650,0.023888,0.442616,0.009018,0.015161,0.002476
5,"{'bootstrap': 'True', 'max_depth': 90, 'n_estimators': 300}",0.930454,0.021927,1.113423,0.123250,0.030518,0.004165
16,"{'bootstrap': 'False', 'max_depth': None, 'n_estimators': 150}",0.930454,0.009026,0.511233,0.006070,0.016157,0.001717
13,"{'bootstrap': 'False', 'max_depth': 90, 'n_estimators': 150}",0.926061,0.018013,0.477323,0.013390,0.014561,0.002720
10,"{'bootstrap': 'False', 'max_depth': 60, 'n_estimators': 150}",0.923865,0.017511,0.498866,0.027082,0.017553,0.002792
4,"{'bootstrap': 'True', 'max_depth': 90, 'n_estimators': 150}",0.922401,0.016190,0.432443,0.008239,0.013564,0.001621


The results are worse than before. The old results are satisfactory so we will use a standard random forest model for now. We may look into oversampling in the future.

## Route Detection:

The words "to" and "from" are words commonly used in carpooling posts. In our dataset the percentage of such posts is given below.

In [12]:
sum([1 for x in derived_posts["stage_3"].tolist() if ("to" or "from") in split_by_comma(x)])/derived_posts.shape[0]

0.9688260543519767

Due to the high occurence of posts with the words "to" and "from", the route detection algorithm uses the relative location of city names in the post, in terms of the words "to" and "from", to determine the origins and destinations of the trips in the post. 

## Duplicate Detection:

It is possible for duplicate messages to appear in the database. One way this can occur is if an individual posts the same post in 2 different Facebook groups. To deal with this, every time a post is read into the database, it is grouped with other duplicate posts up to one week in the past. When searched, only the most recent post in a duplicate group is displayed, along with all Facebook groups in which that post was posted.

## Trips:

It is possible for a post to contain multiple carpools. For example, the following post "Driving from Toronto to Mississauga to Waterloo", contains one carpool from Toronto to Mississauga, another from Mississauga to Waterloo and a final carpool from Toronto to Waterloo. Each such carpool is referred to as a trip. Once a post has been classified, it is mapped to multiple corresponding trips. It is this database of trips that the search engine queries.